In [51]:
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient
from common.trainer import Trainer
from hidden_others.mnist import load_mnist


In [2]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング

    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [3]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [4]:
x2 = np.random.rand(10,3,7,7)
col1 = im2col(x2, 5, 5, stride=1, pad=0)
print(col1.shape)

(90, 75)


##　畳み込み層の実装

In [5]:
class Covalution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FH) / self.stride)  
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

## Pooling layer

In [8]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride= stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_W) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        # チャンネル数分im2colのWが広がるので、reshapeしてあげる
        col = col.reshape(-1, self.pool_h * self.pool_W)
        
        # maxpoolingなので行毎に最大値をとる
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

In [102]:
class SimpleConvNet:
    
    # 初期化と畳み込み層の出力
    def __init__(self,
                 input_dim=(1, 28, 28),
                 conv_param={'filter_num':30,
                              'filter_size':5,
                              'pad':0,
                              'stride':1},
                 hidden_size=100,
                 output_size=10,
                 weight_init_std=0.01):
        
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad / filter_stride + 1) 
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2)) 

        # 重みパラメータの初期化
        self.params = {}
        # (N, C, H, W)
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        
        # layer
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params["W1"],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],
                                        self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],
                                        self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    # 順伝播
    def predict(self, x):
        for layer in self.layers.values():
            # モジュールからロードした関数なのでforwardが使える。
            # xを更新することで、順番にデータをlayerに通すことができる
            x = layer.forward(x)
        return x

    # 損失算出
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)


    # 誤差逆伝播
    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

        

In [103]:
# データのロード
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

In [104]:
max_epochs = 20
network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

train loss:2.3005627384983502
=== epoch:1, train acc:0.291, test acc:0.299 ===
train loss:2.2989081708494106
train loss:2.2936159501707096
train loss:2.289220078320666
train loss:2.2868465768130717
train loss:2.2752143577341513
train loss:2.2612607674686895
train loss:2.248855147199275
train loss:2.2240079618281294
train loss:2.203001484591503
train loss:2.1506626199889367
train loss:2.1363987629541983
train loss:2.096434185726076
train loss:2.0845914451614154
train loss:1.9905571753789542
train loss:1.9875665299072696
train loss:1.9263775061873145
train loss:1.7732024528685195
train loss:1.7317873474536043
train loss:1.63980773778664
train loss:1.4954197385472987
train loss:1.5073391256382302
train loss:1.3767081782898123
train loss:1.2368404584885744
train loss:1.3006159345873491
train loss:1.2092278094393882
train loss:1.0718979122473424
train loss:1.0935972005702923
train loss:0.8597493895451288
train loss:0.9889868883810635
train loss:1.032855085071788
train loss:0.837813438437285